# Procesamiento de Lenguaje
## GPT-muffin

In [135]:
from gensim.models import KeyedVectors
import re
from gensim import downloader
from gensim.models import Word2Vec


1. Cargue los pesos pre-entrenados de 'word2vec-google-news-300'. Puede obtenerlos usando gensim.downloader.load('word2vec-google-news-300') o descargándolos manualmente de otra fuente.

In [136]:
# Cargar el modelo preentrenado word2vec-google-news-300
model = downloader.load('word2vec-google-news-300')

In [137]:
# Encontrar las 10 palabras más similares a 'muffin'
similar_words = model.most_similar('muffin', topn=10)
print("Palabras más similares a 'muffin':")
for word, similarity in similar_words:
    print(f"{word}: {similarity:.4f}")

Palabras más similares a 'muffin':
muffins: 0.6586
blueberry_muffin: 0.6340
hamburger_bun: 0.6315
scone: 0.6308
croissant: 0.6291
oatmeal_cookie: 0.6288
bagel: 0.6188
bran_muffin: 0.6149
red_velvet_cupcake: 0.6067
corn_muffin: 0.6019


2. Cree una función que devuelva la similaridad entre la ultima palabra de una oración y la palabra "muffin" (gensim tiene muchos métodos que pueden ayudar).

In [138]:
# Función para tokenizar una oración y obtener la última palabra
def get_last_word(sentence):
    words = re.findall(r"\w+(?:'\w+)?|[^\w\s]", sentence.lower())
    return words[-1] if words else None

# Función para calcular la similaridad entre la última palabra de una oración y "muffin"
def similarity_with_muffin(sentence, model, target_word='muffin'):
    last_word = get_last_word(sentence)
    if last_word in model:
        similarity = model.similarity(last_word, target_word)
        return similarity, last_word
    else:
        return None

# Ejemplo de uso
sentence = "Last night I ate a vanilla cake"
similarity, ultima_palabra = similarity_with_muffin(sentence, model)

if similarity is not None:
    print(f"La similitud entre '{ultima_palabra}' y 'muffin' es: {similarity:.4f}")
else:
    print("La última palabra no está en el vocabulario del modelo.")


La similitud entre 'cake' y 'muffin' es: 0.4687


3. Proponga e implemente un método para encontrar el umbral de similitud optimo.


In [139]:
# Función para tokenizar una oración y obtener la última palabra
def get_last_word(sentence):
    words = re.findall(r"\w+(?:'\w+)?|[^\w\s]", sentence.lower())
    return words[-1] if words else None

# Definir los targets, uso la palabra en plural como target también ya que los muffins suelen venir en conjunto
targets = ['muffin', 'muffins']
def similarity_with_targets(sentence, model, targets):
    last_word = get_last_word(sentence)
    similarities = []
    if last_word in model: 
        for target in targets:
            similarity = model.similarity(last_word, target)
            similarities.append(similarity)
        return similarities
    else: 
        return None

# Lista de oraciones para probar y si deberían pasar el umbral o no (según la lógica humana)
sentences = [
    "✅ - She loves trying new recipes, now she's experimenting with different types of dessert",
    "✅ - He couldn't resist the smell of a freshly baked cookie",
    "❌ - The festival featured music, games, and plenty of ice cream",
    "✅ - They spent the afternoon shopping for ingredients to bake a decadent cake",
    "❌ - They got toghether to cook a pizza",
    "✅ - She started a blog to share her passion for creating gourmet chocolate",
    "✅ - After a long day at work, she craved comfort food like warm brownies with that one vanilla cupcake",
    "✅ - I wonder if I could make that pasta",
    "✅ - They traveled to different cities to taste famous desserts like New York cheesecake",
    "❌ - That was delic¡ous",
    "✅ - The party had a variety of desserts, including cupcakes and fruit tarts",
    "❌ - You're so sweet"
    ]

In [140]:
#intento con un umbral de 60% de similitud
threshold = 0.6   
# Evaluar cada oración
for sentence in sentences:
    # Calcular similitud con cada target
    similarities = similarity_with_targets(sentence, model, targets)
 # Verificar si alguna similitud supera el umbral
    if any(similarity >= threshold for similarity in similarities):
        print(f"✅Pasa el umbral: {sentence}")
    else:
        print(f"❌No pasa el umbral: {sentence}")

❌No pasa el umbral: ✅ - She loves trying new recipes, now she's experimenting with different types of dessert
❌No pasa el umbral: ✅ - He couldn't resist the smell of a freshly baked cookie
❌No pasa el umbral: ❌ - The festival featured music, games, and plenty of ice cream
❌No pasa el umbral: ✅ - They spent the afternoon shopping for ingredients to bake a decadent cake
❌No pasa el umbral: ❌ - They got toghether to cook a pizza
❌No pasa el umbral: ✅ - She started a blog to share her passion for creating gourmet chocolate
❌No pasa el umbral: ✅ - After a long day at work, she craved comfort food like warm brownies with that one vanilla cupcake
❌No pasa el umbral: ✅ - I wonder if I could make that pasta
✅Pasa el umbral: ✅ - They traveled to different cities to taste famous desserts like New York cheesecake
❌No pasa el umbral: ❌ - That was delic¡ous
❌No pasa el umbral: ✅ - The party had a variety of desserts, including cupcakes and fruit tarts
❌No pasa el umbral: ❌ - You're so sweet


In [141]:
#como rechazó la mayoría de las palabras finales intento con un umbral de 50% de similitud
threshold = 0.5   
# Evaluar cada oración
for sentence in sentences:
    # Calcular similitud con cada target
    similarities = similarity_with_targets(sentence, model, targets)
 # Verificar si alguna similitud supera el umbral
    if any(similarity >= threshold for similarity in similarities):
        print(f"✅Pasa el umbral: {sentence}")
    else:
        print(f"❌No pasa el umbral: {sentence}")

✅Pasa el umbral: ✅ - She loves trying new recipes, now she's experimenting with different types of dessert
✅Pasa el umbral: ✅ - He couldn't resist the smell of a freshly baked cookie
❌No pasa el umbral: ❌ - The festival featured music, games, and plenty of ice cream
❌No pasa el umbral: ✅ - They spent the afternoon shopping for ingredients to bake a decadent cake
❌No pasa el umbral: ❌ - They got toghether to cook a pizza
✅Pasa el umbral: ✅ - She started a blog to share her passion for creating gourmet chocolate
✅Pasa el umbral: ✅ - After a long day at work, she craved comfort food like warm brownies with that one vanilla cupcake
❌No pasa el umbral: ✅ - I wonder if I could make that pasta
✅Pasa el umbral: ✅ - They traveled to different cities to taste famous desserts like New York cheesecake
❌No pasa el umbral: ❌ - That was delic¡ous
✅Pasa el umbral: ✅ - The party had a variety of desserts, including cupcakes and fruit tarts
❌No pasa el umbral: ❌ - You're so sweet


In [142]:
#intento con un umbral de 40% de similitud
threshold = 0.4   
# Evaluar cada oración
for sentence in sentences:
    # Calcular similitud con cada target
    similarities = similarity_with_targets(sentence, model, targets)
 # Verificar si alguna similitud supera el umbral
    if any(similarity >= threshold for similarity in similarities):
        print(f"✅Pasa el umbral: {sentence}")
    else:
        print(f"❌No pasa el umbral: {sentence}")

✅Pasa el umbral: ✅ - She loves trying new recipes, now she's experimenting with different types of dessert
✅Pasa el umbral: ✅ - He couldn't resist the smell of a freshly baked cookie
❌No pasa el umbral: ❌ - The festival featured music, games, and plenty of ice cream
✅Pasa el umbral: ✅ - They spent the afternoon shopping for ingredients to bake a decadent cake
✅Pasa el umbral: ❌ - They got toghether to cook a pizza
✅Pasa el umbral: ✅ - She started a blog to share her passion for creating gourmet chocolate
✅Pasa el umbral: ✅ - After a long day at work, she craved comfort food like warm brownies with that one vanilla cupcake
✅Pasa el umbral: ✅ - I wonder if I could make that pasta
✅Pasa el umbral: ✅ - They traveled to different cities to taste famous desserts like New York cheesecake
❌No pasa el umbral: ❌ - That was delic¡ous
✅Pasa el umbral: ✅ - The party had a variety of desserts, including cupcakes and fruit tarts
❌No pasa el umbral: ❌ - You're so sweet


Como con el 0.4 se aceptan cosas que no deberian y en el 0.5 se rechazan las que sí son aceptables intento con un umbral de 45% de similitud


In [143]:
threshold = 0.45 
# Evaluar cada oración
for sentence in sentences:
    # Calcular similitud con cada target
    similarities = similarity_with_targets(sentence, model, targets)
 # Verificar si alguna similitud supera el umbral
    if any(similarity >= threshold for similarity in similarities):
        print(f"✅Pasa el umbral: {sentence}")
    else:
        print(f"❌No pasa el umbral: {sentence}")

✅Pasa el umbral: ✅ - She loves trying new recipes, now she's experimenting with different types of dessert
✅Pasa el umbral: ✅ - He couldn't resist the smell of a freshly baked cookie
❌No pasa el umbral: ❌ - The festival featured music, games, and plenty of ice cream
✅Pasa el umbral: ✅ - They spent the afternoon shopping for ingredients to bake a decadent cake
❌No pasa el umbral: ❌ - They got toghether to cook a pizza
✅Pasa el umbral: ✅ - She started a blog to share her passion for creating gourmet chocolate
✅Pasa el umbral: ✅ - After a long day at work, she craved comfort food like warm brownies with that one vanilla cupcake
✅Pasa el umbral: ✅ - I wonder if I could make that pasta
✅Pasa el umbral: ✅ - They traveled to different cities to taste famous desserts like New York cheesecake
❌No pasa el umbral: ❌ - That was delic¡ous
✅Pasa el umbral: ✅ - The party had a variety of desserts, including cupcakes and fruit tarts
❌No pasa el umbral: ❌ - You're so sweet


Puede que no sea la manera más óptima de encontrar el umrbal pero con el prueba y error se determina que el umbral de similitud debería superar el 0.45 o 45% para sea correcto cambair la últimá palabra por 'muffin' o 'muffins'

4. Cree una función que devuelva 'True' si la última palabra de una oración puede ser reemplazada por 'muffin'.s


In [144]:
sentences = [
    "✅ - She loves trying new recipes, now she's experimenting with different types of dessert",
    "✅ - He couldn't resist the smell of a freshly baked cookie",
    "❌ - The festival featured music, games, and plenty of ice cream",
    "✅ - They spent the afternoon shopping for ingredients to bake a decadent cake",
    "❌ - They got toghether to cook a pizza",
    "✅ - She started a blog to share her passion for creating gourmet chocolate",
    "✅ - After a long day at work, she craved comfort food like warm brownies with that one vanilla cupcake",
    "✅ - I wonder if I could make that pasta",
    "✅ - They traveled to different cities to taste famous desserts like New York cheesecake",
    "❌ - That was delic¡ous",
    "✅ - The party had a variety of desserts, including cupcakes and fruit tarts",
    "❌ - You're so sweet"
    ]

# Función para tokenizar una oración y obtener la última palabra
def get_last_word(sentence):
    words = re.findall(r"\w+(?:'\w+)?|[^\w\s]", sentence.lower())
    return words[-1] if words else None

# Definir los targets, uso la palabra en plural como target también ya que los muffins suelen venir en conjunto
targets = ['muffin', 'muffins']
def similarity_with_targets(sentence, model, targets):
    last_word = get_last_word(sentence)
    similarities = []
    if last_word in model: 
        for target in targets:
            similarity = model.similarity(last_word, target)
            similarities.append(similarity)
        return similarities
    else: 
        return None
    
threshold = 0.45 

for sentence in sentences:
    similarities = similarity_with_targets(sentence, model, targets)
    if any(similarity >= threshold for similarity in similarities):
            print("True")
    else:
            print("False")


True
True
False
True
False
True
True
True
True
False
True
False


5. Testee su modelo con las siguientes oraciones:

In [145]:
sentences = [
"I'm going to eat a tasty cupcake", #-> True
"I was driving my car", #-> False
"The tree is green", #-> False
"It was delicious to eat that cake", #-> True
"This is not arborio rice it is carnaroli rice", #-> ?
"Inside the box there was a cat", #-> ?
"Uhh, it's pronounced bagel", #-> ?, #-> ?
]


# Función para tokenizar una oración y obtener la última palabra
def get_last_word(sentence):
    words = re.findall(r"\w+(?:'\w+)?|[^\w\s]", sentence.lower())
    return words[-1] if words else None

# Definir los targets, uso la palabra en plural como target también ya que los muffins suelen venir en conjunto
targets = ['muffin', 'muffins']
def similarity_with_targets(sentence, model, targets):
    last_word = get_last_word(sentence)
    similarities = []
    if last_word in model: 
        for target in targets:
            similarity = model.similarity(last_word, target)
            similarities.append(similarity)
        return similarities
    else: 
        return None
    
threshold = 0.45 

for sentence in sentences:
    similarities = similarity_with_targets(sentence, model, targets)
    if any(similarity >= threshold for similarity in similarities):
            print(f"{sentence} - True")
    else:
            print(f"{sentence} - False")

I'm going to eat a tasty cupcake - True
I was driving my car - False
The tree is green - False
It was delicious to eat that cake - True
This is not arborio rice it is carnaroli rice - False
Inside the box there was a cat - False
Uhh, it's pronounced bagel - True


6. ¿Qué piensa acerca de los casos con signo de interrogación? ¿Qué podría hacer para mejorar su modelo?
* "This is not arborio rice it is carnaroli rice" -> ?
* "Inside the box there was a cat" -> ?
* "Uhh, it's pronounced bagel" -> ?
* "t is the last letter of the word croissant" -> ?


En estas cuatro oraciones es notable como la última palabra, aún si se reemplaza por '*muffin*', ya que su coherencia depende del contexto, del significado de la oración. No se pueden interambiar palabras por otras únicamente porque pertenecen a la misma clase o tienen vectores parecidos. Aquí va una explicación con los siguientes contraejemplos: 

In [151]:
similarity_1 = model.similarity('rice', 'corn')
print (f"Similaridad entre 'rice' y 'corn': {similarity_1} ")

similarity_2 = model.similarity('cat', 'elephant')
print (f"Similaridad entre 'cat' y 'elephant': {similarity_2} ")

similarity_3 = model.similarity('bagel', 'donut')
print (f"Similaridad entre 'bagel' y 'donut': {similarity_3} ")

similarity_4 = model.similarity('croissant', 'crêpes')
print (f"Similaridad entre 'croissant' y 'crêpes': {similarity_4} ")

similarity_5 = model.similarity('croissant', 'net')
print (f"Similaridad entre 'croissant' y 'net': {similarity_5} ")

Similaridad entre 'rice' y 'corn': 0.6120111346244812 
Similaridad entre 'cat' y 'elephant': 0.4638771414756775 
Similaridad entre 'bagel' y 'donut': 0.6141875386238098 
Similaridad entre 'croissant' y 'crêpes': 0.6461926102638245 
Similaridad entre 'croissant' y 'net': 0.0484132394194603 


En el caso de la oración **"This is not arborio rice it is carnaroli *rice*"**, según el modelo entrenado y con el umbral de similidad establecido ($0.45$), se podría aceptar con certidumbre que puede reemplazarse la palabra *rice* (arroz) por *corn* (maíz), ya que su similaridad es de $0.61$ (aceptable por el umbral). Pero la oración habla de diferentes tipos de arroz, no de maíz. Entonces no es coherente decir que "This is not arborio rice it is carnaroli *corn*", porque no existe el maíz carnaroli. No se le puede atribuir el adjetivo/clasificación porque no le pertenece.


En la segunda oración, también ocurre una ambiguedad si se intercambia la palabra *cat* (gato) por *elephant* (elefante), aún si su similaridad es aceptada por el umbral. El mensaje que dice que "Dentro de la caja hay un **gato**" puede tener sentido, mientras que el de "Dentro de la caja hay un **elefante**" carece completamente de este. La semántica de las palabras puede ser parecida pero no se debe dejar de lado el sentido de la oración.


El tercer caso, la oración **"Uhh, it's pronounced bagel"**, si se le reemplaza la última palabra por cualquier otra podría tener sentido, si sucede en una conversación que no mantiene un hilo razonable. Ya que si alguien pronuncia mal la palabra *bagel*, no sería lógico que se le corriga diciendo que "Se pronuncia *donut*"; aún si ambas palabras tienen vectores muy parecidos (similaridad de $0.61$). La oración se refiere directamente a la palabra, no se la puede reemplazar. 

Finalmente, en el último caso, ocurre similar que el caso del *bagel*; la oración habla específicamente de la palabra final, de su ortografía. Entonces, no tiene sentido cambiar la palabra por una que se utiliza en contextos similares, como lo es la pastelería; sino que se debería buscar la similitud ortográfica, porque de eso habla la oración. Si las palabras, semanticamente son similares, como es este caso, van a tener una *similarity* alta, pero la oración no va a tener sentido. La palabra *net* (red) tiene una similaridad menor al $0.05$ con *bagel*, pero en el contexto de la oración está mejor colocada que *donut*, de similaridad semántica de $0.65$ con *bagel*.